In [82]:
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet18
import torch
import albumentations as A
from albumentations import *
from albumentations.pytorch import ToTensorV2
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import torchvision

In [95]:
data_dir = "./dataset/train/"
def get_img_tensor(file_name):
    transforms = A.Compose([
                    A.Resize(always_apply=False, p=1.0, height=540, width=960, interpolation=0),
#                     A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#                     ToTensorV2()
                                ])
    img = cv2.imread(data_dir + file_name)
    img = transforms(image=img)["image"]
    img = Image.fromarray(img[:, :, :3]) # 여기를 수정 해줘야함.
    img = torchvision.transforms.Resize((540, 960))(img)
    img = torchvision.transforms.ToTensor()(img)
    img = torchvision.transforms.Normalize(mean = (0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225))(img)
    img = img.unsqueeze(0)
    return img
    

In [96]:
model = resnet18()
device = torch.device('cpu')
model.load_state_dict(torch.load("./models/test_model_epoch33.pth",map_location=device))

<All keys matched successfully>

In [261]:
model.layer4[-1]

BasicBlock(
  (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [268]:
target_layers = [model.layer4[-1]]
cam = GradCAM(model=model, target_layers=target_layers, use_cuda=False)

In [294]:
img_name = "남산타워.jpeg"
img = get_img_tensor(img_name)
image = plt.imread(f"./dataset/train/{img_name}")

In [295]:
pred = int(torch.argmax(torch.nn.Softmax()(model(img))).item())

/var/folders/r3/mrxww_xx159_m47pkh4ks07r0000gn/T/ipykernel_43604/1315154292.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = int(torch.argmax(torch.nn.Softmax()(model(img))).item())


In [296]:
pred

6

In [297]:
targets = [ClassifierOutputTarget(pred)]
grayscale_cam = cam(input_tensor=img, targets=targets)
grayscale_cam = grayscale_cam[0, :]
visualization = show_cam_on_image(image, grayscale_cam, use_rgb=True)

Exception: The input image should np.float32 in the range [0, 1]

In [ ]:
plt.imshow(visualization)